# [실습] 벡터 데이터베이스 기반 RAG 어플리케이션

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로, 질문이 주어지면 관련 있는 문서를 찾아 프롬프트에 추가하는 방식의 어플리케이션입니다.   
RAG의 과정은 아래와 같이 진행됩니다.
1. Indexing : 문서를 받아 검색이 잘 되도록 저장합니다.
1. Processing : 입력 쿼리를 전처리하여 검색에 적절한 형태로 변환합니다<br>(여기서는 수행하지 않습니다)
1. Search(Retrieval) : 질문이 주어진 상황에서 가장 필요한 참고자료를 검색합니다.
1. Augmenting : Retrieval의 결과와 입력 프롬프트를 이용해 LLM에 전달할 프롬프트를 생성합니다.
1. Generation : LLM이 출력을 생성합니다.

In [1]:
#!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 KB 8.5 MB/s eta 0:00:0000:0100:01
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Attempting uninstall: chroma-hnswlib
    Found existing installation: chroma-hnswlib 0.7.6
    Uninstalling chroma-hnswlib-0.7.6:
      Successfully uninstalled chroma-hnswlib-0.7.6
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.6.3
    Uninstalling chromadb-0.6.3:
      Successfully uninstalled chromadb-0.6.3


In [1]:
import os
import json

with open("api_key.json", "r") as f:
    config = json.load(f)

api_key = config["OPENAI_API_KEY"]

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY'] = api_key

os.environ['USER_AGENT']='MyCustomAgent'
# 아래 코드 Warning 제거

## 1. `WebBaseLoader`로 웹 페이지 받아오기

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있습니다.   
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

이번에는 웹 페이지를 로드하는 `WebBaseLoader`를 통해 뉴스 기사를 읽어보겠습니다.    
- 최근에는 FireCrawl(https://www.firecrawl.dev/)을 사용하는 경우가 늘고 있습니다.

네이버 API를 사용해, 네이버 뉴스 검색 링크를 가져옵니다.

In [2]:
# 스포츠 뉴스는 형식이 달라서 지원하지 않습니다...

import requests
def get_naver_news_links(query, num_links=100):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
        'X-Naver-Client-Secret': 'BvqX8mNtHu'
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    # 결과 출력
    print(len(filtered_links))
    for link in filtered_links:
        print(link)
    return filtered_links

filtered_links = []
for topic in ['LLM', '생성 인공지능', 'GPT', '딥러닝', '가전제품']:
    filtered_links += get_naver_news_links(topic, 100)
print(len(filtered_links))
print(len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

57
https://n.news.naver.com/mnews/article/293/0000064488?sid=105
https://n.news.naver.com/mnews/article/037/0000035830?sid=105
https://n.news.naver.com/mnews/article/277/0005554480?sid=105
https://n.news.naver.com/mnews/article/015/0005101683?sid=105
https://n.news.naver.com/mnews/article/001/0015243264?sid=105
https://n.news.naver.com/mnews/article/003/0013096548?sid=105
https://n.news.naver.com/mnews/article/092/0002365544?sid=105
https://n.news.naver.com/mnews/article/293/0000064445?sid=105
https://n.news.naver.com/mnews/article/014/0005316599?sid=101
https://n.news.naver.com/mnews/article/421/0008112578?sid=105
https://n.news.naver.com/mnews/article/003/0013097927?sid=105
https://n.news.naver.com/mnews/article/031/0000912965?sid=105
https://n.news.naver.com/mnews/article/417/0001061403?sid=105
https://n.news.naver.com/mnews/article/029/0002939041?sid=102
https://n.news.naver.com/mnews/article/030/0003289256?sid=105
https://n.news.naver.com/mnews/article/018/0005954203?sid=105
https

WebBaseLoader를 이용해, 링크로부터 본문을 불러옵니다.

In [9]:
# 주의: 동일 IP 환경에서 동시에 다수 실행하면 차단의 위험이 있음

import bs4
from langchain_community.document_loaders import WebBaseLoader
def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("newsct", "newsct-body")
                # newsct, newsct-body만 추출 : 도메인마다 다름
            )
        ),
        requests_per_second = 1, # 1초에 1개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    docs = loader.load()
    len(docs)
    return docs
docs = get_news_documents(filtered_links[:5])

KeyboardInterrupt: 

In [8]:
docs

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/016/0002437252?sid=105'}, page_content='\n\n\n\n\n헤럴드경제\n\n헤럴드경제\n\n\n구독\n\n헤럴드경제 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n헤럴드경제 언론사 구독 해지되었습니다.\n\n\n\n\nKIRD, ‘생성형 AI’ 실전 활용 노하우 전수\n\n\n\n\n입력2025.03.05. 오전 10:55\n\n기사원문\n \n\n\n\n\n구본혁 기자\n\nTALK\n\n\n\n\n\n\n\n구본혁 기자\n\n\n\n\n구본혁 기자\n\nTALK\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n- 교육기획자와 교수자 위한 생성형 AI 활

In [5]:
print(docs[0:4])

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/003/0013092974?sid=105'}, page_content='\n\n\n\n\n뉴시스\n\n뉴시스\n\n\n구독\n\n뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴시스 언론사 구독 해지되었습니다.\n\n\n\n\n챗GPT, 유료서비스도 한도 제한?…방통위, 법 위반 여부 살핀다(종합)\n\n\n\n\n입력2025.02.27. 오후 5:15\n\n\n수정2025.02.27. 오후 5:16\n\n기사원문\n \n\n윤현성 기자\n박은비 기자\n\n\n\n\n\n\n\n\n윤현성 기자\n\n\n\n\n윤현성 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n박은비 기자\n\n\n\n\n박은비 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n

In [6]:
# 참고) 불러온 document 저장하기

import jsonlines
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())
save_docs_to_jsonl(docs, "docs.jsonl")

# 참고) jsonl 파일 불러오기
from langchain.schema import Document

def load_docs_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_51540/3672484833.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  writer.write(doc.dict())


In [7]:
# 파일로부터 다시 불러오기
docs = load_docs_from_jsonl("docs.jsonl")
len(docs)

281

In [8]:
docs[0].page_content

'\n\n\n\n\n뉴시스\n\n뉴시스\n\n\n구독\n\n뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴시스 언론사 구독 해지되었습니다.\n\n\n\n\n챗GPT, 유료서비스도 한도 제한?…방통위, 법 위반 여부 살핀다(종합)\n\n\n\n\n입력2025.02.27. 오후 5:15\n\n\n수정2025.02.27. 오후 5:16\n\n기사원문\n \n\n윤현성 기자\n박은비 기자\n\n\n\n\n\n\n\n\n윤현성 기자\n\n\n\n\n윤현성 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n박은비 기자\n\n\n\n\n박은비 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n방통위, 챗GPT 유료서비스 대상 사실조사…

불러온 뉴스기사에는 불필요한 내용이 다소 존재합니다.    
아래의 과정을 통해 간단하게 전처리합니다.

In [9]:
def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',


    ]

    def clean_text(text):
        # 잡음을 제거하고 여러 공백을 하나로 줄이는 함수
        text = text.replace('\t',' ').replace('\n',' ')
        for _ in range(20):
            text = ' '.join(text.split())  # 연속된 공백을 하나로
        for noise in noise_texts:
            text = text.replace(noise, '')

        return text

    preprocessed_docs = []
    for doc in docs:
        try:
            # 텍스트 자르기
            content = doc.page_content.split('구독 해지되었습니다.')[1]
        except:
            # 구독 관련 정보가 없거나 텍스트 처리 문제일 경우 원본 텍스트 유지
            content = doc.page_content
        try:
            content = doc.page_content.split('구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!')[0]
        except:
            content = doc.page_content


        # 텍스트 정제 작업
        content = clean_text(content)
        doc.page_content = content
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


In [10]:
preprocessed_docs[10]


Document(metadata={'source': 'https://n.news.naver.com/mnews/article/022/0004015819?sid=101'}, page_content='세계일보 세계일보 구독 세계일보 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 세계일보 언론사 구독 해지되었습니다. PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기 CCTV에 챗GPT 탑재…사전 위험 안내해 사고 예방 입력2025.03.04. 오후 1:19 기사원문 이진경 기자 이진경 기자 이진경 기자 구독  추천      한국스마트안전보건협회(한스협)는 폐쇄회로(CC)TV에 생성형 인공지능(AI) 챗GPT를 접목한 ‘사고 위험 AI 알람 시스템’을 출시했다고 4일 밝혔다. AI CCTV 시스템은 CCTV를 통해 강풍이나 위험구역 접근 등 유해 요인을 감지하고 이를 즉각 관리자에게 보고해 사고 예방을 위한 즉각적 조치를 하도록 지원한다. 건설, 제조, 중공업 등의 분야에서 활용이 가능하다. 이 시스템은 클라우드 기반으로, 기존 CCTV를 활용해 저비용으로 고성능 AI 영상 분석이 가능하다. 기존 지능형 CCTV만으로 실시간 영상 분석을 하려면 별도 서버 설치가 필수적이어서 많은 구축 비용과 유지보수 비용이 발생한다. 외국인 근로자를 위한 다국어 안전관리 기능도 탑재해 위험 발생 시 중국, 베트남 등 14개 외국어로 즉각적으로 음성안내를 한다. 이를 통해 위험성 평가와 TBM(작업 전 안전점검회의), 안전방송 등 내용을 실시간 공유할 수 있다. 안전 보고서도 작성해준다. 실시간 AI 사고위험 보고서와 현장 작업 보고서 등을 특정 시간을 설정해 받아볼 수 있다. 또 작업 현황과 안전위반 사례 등이 담긴 일일 요약보고서를 생성하고, 중대재해처벌법 등에 따른 사고 전후 초동 보고서도 지원한다. 여러 현장을 동시에 관리할 경우 현장별로 일정 시간에 맞춰 현장 단위 보고서를 생성하도록 설정해 효율적으로 운용할 수도 있다. 

## 2. Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# 0~2000, 1800~3800, 3600~5600, ...
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

559


구성된 청크를 벡터 데이터베이스에 로드합니다.   
`Chroma.from_documents`는 documents의 임베딩을 구하고 이를 DB에 저장합니다.

In [12]:
chunks[110].page_content

'다녀요"…부업으로 2000만원 이상 번 사람 무려 발열·구토 등 증상 발현 후 48시간내 사망…치사율 12% 괴질의 정체 이 기사를 추천합니다 기사 추천은 24시간 내 50회까지 참여할 수 있습니다. 닫기  모두에게 보여주고 싶은 기사라면?beta 이 기사를 추천합니다 버튼을 눌러주세요. 집계 기간 동안 추천을 많이 받은 기사는 네이버 자동 기사배열 영역에 추천 요소로 활용됩니다. 레이어 닫기 아시아경제 언론사가 직접 선정한 이슈 이슈 대선, 몸풀기 시작 김문수 “尹 정부, 교육시설 기관에 친윤 ‘알박기’” 이슈 尹대통령 탄핵심판 김문수 “尹 정부, 교육시설 기관에 친윤 ‘알박기’” 이슈 트럼프 2.0 시대 트럼프 관세에 美 차값 최대 1800만원 오른다 이슈 불황 한파 생존기 생산·소비·투자 \'트리플\' 감소… 생산, 팬데믹 이후 최대폭(종합) 이전 다음 아시아경제 언론사홈 바로가기 기자 구독 후 기사보기 구독 없이 계속 보기'

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [20]:
Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

# 이미 존재하는 DB에서 가져오기
# db = Chroma(
#      embedding=OpenAIEmbeddings(model='text-embedding-3-large'),
#      large: 3072차원, small: 1536차원
#      persist_directory="./chroma_Web"
#      )
# Collection 이름 없이 생성한 경우, 기본 Collection으로 저장된 경우

db = Chroma.from_documents(documents=chunks,
                           embedding=OpenAIEmbeddings(model='text-embedding-3-small'), # text-embedding-3-large
                           # large: 3072차원, small: 1536차원
                           persist_directory="./chroma_Web", # ./는 current directory를 의미
                           # persist_directory를 쓰지 않으면 메모리에 저장
                           collection_metadata={'hnsw:space':'l2'}
                           # l2 메트릭 설정(기본값)
                           # cosine, mmr
                           )
# Collection 이름 없이 생성한 경우, 기본 Collection으로 저장됨

retriever는 query에 맞춰 db에서 문서를 검색합니다.

In [21]:
retriever = db.as_retriever()

In [22]:
retriever.invoke("도메인 특화 LLM에 대한 소식 있어?")

[Document(id='7ee6340b-65b0-4f17-970d-ca28436b08b5', metadata={'source': 'https://n.news.naver.com/mnews/article/003/0013084396?sid=102'}, page_content='뉴시스 뉴시스 구독 뉴시스 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 뉴시스 언론사 구독 해지되었습니다. PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기 챗GPT에 손쉽게 개인정보 뚫린다…KAIST, LLM 악용 가능성 입증 입력2025.02.24. 오후 1:53 수정2025.02.24. 오후 2:27 기사원문 김양수 기자 김양수 기자 김양수 기자 구독  추천      신승원·이기민 교수팀, 대형언어모델의 정보수집·피싱 공격 규명최대 95.9% 정확도로 개인정보 수집, 피싱 이메일 클릭률 46.67%까지 증가저비용으로 개인정보 탈취해 공격 가능…국제학술지 게재 [대전=뉴시스] KAIST가 챗GPT 등 대형언어모델(LLM)을 활용한 개인정보 수집 및 피싱공격 실험을 수행해 LLM 에이전트가 손쉽게 개인정보 수집 및 피싱공격 등을 할 수 있다는 사실을 입증했다.(윗줄 왼쪽부터)KAIST 전기및전자공학부 나승호 박사, 김재철AI대학원 이기민 교수.(아랫줄 왼쪽부터)전기및전자공학부 김한나 박사과정, 신승원 교수, 송민규 박사과정.(사진=KAIST 제공) *재판매 및 DB 금지[대전=뉴시스] 김양수 기자 = 챗GPT로 손쉽게 개인정보를 수집하고 악용할 수 있다는 연구결과가 국내연구진에 의해 발표됐다.한국과학기술원(KAIST)은 전기및전자공학부 신승원 교수, 김재철 AI 대학원 이기민 교수 공동연구팀이 실제환경에서 챗GPT와 같은 대형 언어모델(LLM)이 사이버공격에 악용될 가능성을 실험적으로 규명했다고 24일 밝혔다.최근 인공지능 기술의 발전으로 LLM은 단순한 챗봇을 넘어 자율적인 에이전트로 발전하고 있다. 현재 OpenAI, 구글 AI 

## 3. Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print() # pretty_print() 사람이 보기 편하게 출력

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


## 4. Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
체인의 입력은 Question만 들어가므로, Context를 동시에 prompt에 넣기 위해서는 아래의 구성이 필요합니다.

In [25]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return "\n\n---\n\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결
    # Ex)'와'.join(['a','b','c']) = 'a와b와c'

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
rag_chain.invoke("도메인 특화 LLM이 뭔가요?")

'도메인 특화 LLM(대형 언어 모델)은 특정 분야나 산업에 최적화된 인공지능 언어 모델을 의미합니다. 이러한 모델은 일반적인 LLM과 달리 특정 조직이나 기업의 요구에 맞춰 설계되어, 해당 분야의 전문 데이터를 학습하여 보다 정교하고 신뢰성 높은 정보를 생성할 수 있습니다. 예를 들어, 금융 산업에 특화된 LLM은 금융 관련 용어와 개념을 잘 이해하고, 고객의 질문에 대해 정확한 답변을 제공하는 데 중점을 둡니다. 이러한 특화된 모델은 보안성이 뛰어나고, 특정 도메인에서의 성능을 극대화하는 데 기여합니다.'

In [27]:
rag_chain.invoke("오픈 AI 최근 소식 있어?")

"오픈AI는 최근 'GPT-4.5'라는 새로운 인공지능 모델을 공개했습니다. 이 모델은 추론과 증류 기능이 없는 마지막 일반 모델로, 특정 사용자 그룹에 제한적으로 제공되며, 월 200달러의 '챗GPT 프로' 요금제 사용자에게 우선 제공됩니다. GPT-4.5는 이전 모델들보다 더 높은 정확도와 감성지능을 갖추고 있어, 보다 자연스러운 대화가 가능하다고 평가받고 있습니다. 또한, 오픈AI는 향후 GPT-5에서 추론 모델과 일반 모델을 통합할 계획이라고 밝혔습니다."

In [28]:
rag_chain.invoke("한국에서 만든 초거대 언어 모델은 뭐가 있나요? 참고 링크도 올려주세요")

'정보가 부족하여 답변할 수 없습니다.'

In [29]:
rag_chain.invoke("sllm이 뭐야?")

'정보가 부족하여 답변할 수 없습니다.'

만약 Context가 포함된 RAG 결과를 보고 싶다면, RunnableParallel을 사용하면 됩니다.

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [30]:
from langchain_core.runnables import RunnableParallel
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("엔비디아에 대한 소식은?")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에

{'context': 'KBS KBS 구독 KBS 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 KBS 언론사 구독 해지되었습니다. 이재명 “모든 국민에 인공지능 활용법 가르쳐야”…‘엔비디아’ 발언 비판에 “문맹 수준” 입력2025.03.04. 오전 9:47 수정2025.03.04. 오전 10:52 기사원문 손서영 기자 손서영 기자 손서영 기자 구독  추천      더불어민주당 이재명 대표는 "AI(인공지능)가 일상생활은 물론 학습, 연구, 개발 등에 없어서는 안 될 필수 도구가 될 것"이라며 모든 국민에게 활용법을 교육할 필요가 있다고 밝혔습니다. 민주당 이재명 대표는 오늘(4일) SNS를 통해 "인공지능이 지배적 기술이 되는 어느 시점인가는 기본적 인프라로 모든 국민이 최소한의 인공지능 이용 서비스를 제공받게 될 것"이라며 "그 시점을 하루라도 앞당기고 미리 준비하는 것이 대한민국의 밝은 미래를 여는 것"이라고 말했습니다. 그러면서 "모든 국민들에게 무상 의무교육을 시켜 한글과 산수, 기초 교양을 가르치는 것처럼 모든 국민에게 인공지능 활용법을 가르쳐야 한다"고 강조했습니다. 이 대표는 또 "교과서를 무상 제공한 것처럼 인공지능 이용 기회를 부여해야 할 것"이라며 "대한민국의 데이터를 모으고 국가 차원의 소버린 AI 체계 구축도 해야 한다"고 말했습니다. 이 대표는 "저는 자료 조사나 분석을 보좌진보다 챗지피티에 더 의존한다"며 "지속적으로 AI 성능도 개선될 것이고 의존도는 더 올라갈 것"이라고 밝혔습니다. 이어 "생성형 인공지능 챗지피티의 월간 이용료는 현재 20불이고 더 내릴 것"이라며 "보안 문제로 사용이 제한되었지만 챗지피티보다 성능이 유사하거나 더 낫다는 중국 딥시크는 아예 무료다, 딥시크는 개발비는 매우 적고 최첨단 GPU를 사용하지도 않았다고 한다"고 말했습니다. ■ \'K-엔비디아\' 여권 비판에 "극우 본색에 문맹 수준 식견" 한편 이 대표는 이른바 \'K-엔비디아\' 구상과 관련해 여권을 중심으로 \'반기업

In [31]:
# Runnable Quiz

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})
# 결과 이해해보기!

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## Gradio로 배포하기

입력과 출력을 통해 RAG를 수행합니다.   
크롤링을 수행하는 것은 시간이 오래 걸리기 때문에,     
기본값을 5로 두고 수정 가능하게 만들어 보겠습니다.

In [32]:
def construct_vector_db(query, num_links=5):
    filtered_links = get_naver_news_links(query, num_links)
    docs = get_news_documents(filtered_links)
    preprocessed_docs = preprocess(docs)
    chunks = text_splitter.split_documents(preprocessed_docs)
    Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

    db = Chroma.from_documents(documents=chunks,
                               embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
                               collection_metadata={'hnsw:space':'l2'})
    return db

def RAG(query, question, num_links=5):
    db = construct_vector_db(query, num_links)
    retriever = db.as_retriever()
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)


In [33]:
!pip install gradio

  Obtaining dependency information for markupsafe~=2.0 from https://files.pythonhosted.org/packages/11/e7/291e55127bb2ae67c64d66cef01432b5933859dfb7d6949daa721b89d0b3/MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl (18 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langflow 1.0.18 requires certifi<2025.0.0,>=2023.11.17, but you have certifi 2025.1.31 which is incompatible.
langflow 1.0.18 requires chromadb<0.5,>=0.4, but you have chromadb 0.5.3 which is incompatible.
langflow 1.0.18 requires crewai[tools]<0.37.0,>=0.36.

In [34]:
import gradio as gr

def gradio_rag_interface(query, question, num_links):
    try:
        response = RAG(query, question, num_links)
        return response
    except Exception as e:
        return f"에러 발생: {str(e)}"

with gr.Blocks() as demo:
    gr.Markdown("# 🔍 네이버 뉴스용 검색 증강 생성(RAG)")
    gr.Markdown(
        """
        네이버 뉴스와 관련된 쿼리를 입력하면, 시스템이 관련 기사를 검색,
        처리하고 검색된 정보를 기반으로 종합적인 답변을 생성합니다.
        """
    )

    with gr.Row():
        query_input = gr.Textbox(
            label="📄 쿼리 입력",
            placeholder="예: 최신 AI 기술 발전",
            lines=2
        )
        num_links_input = gr.Slider(
            minimum=1,
            maximum=20,
            step=1,
            value=5,
            label="🔗 뉴스 링크 수",
            info="검색 및 처리할 뉴스 기사의 수를 선택하세요."
        )
        question_input = gr.Textbox(
            label="❓ 질문 입력",
            placeholder="예: 퓨리오사 AI는 어떤 일을 했나요?",
            lines=2
        )

    submit_button = gr.Button("✅ 답변 생성")

    output_box = gr.Textbox(
        label="📝 생성된 답변",
        placeholder="답변이 여기에 표시됩니다...",
        lines=10
    )

    # 버튼 클릭 이벤트 정의
    submit_button.click(
        fn=gradio_rag_interface,
        inputs=[query_input, question_input, num_links_input],
        outputs=output_box
    )

    gr.Markdown(
        """
        ---
        [Gradio](https://gradio.app/) 및 OpenAI의 GPT 모델을 사용하여 제작되었습니다.
        """
    )


demo.launch(server_name="0.0.0.0", server_port=8080)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
